In [5]:
!pip install bs4
from bs4 import BeautifulSoup
import requests
import pandas as pd

Taken a random url from wikipedia that has the necessary data

In [6]:
url="https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&direction=prev&oldid=926287641"

In [33]:
data=requests.get(url).text
soup=BeautifulSoup(data,"html5lib")
tables=soup.find('table')

Create a dataframe to contain the required data. 

In [42]:
neighborhood_data = pd.DataFrame(columns=["PostalCode","Borough","Neighborhood"])

lastpostalcode=""
lastborough=""
lastneighborhood=""
for row in soup.find("tbody").find_all("tr"):
    
    col = row.find_all("td")
    
    postalcode=""
    borough=""
    neighborhood=""
    
    #If col is not populated then no need to populate the values in the data frame - so loop back
    if (col == []):
        continue
        
    #Assign the current col values to appropriate variables
    postalcode = col[0].text
    borough = col[1].text.strip()
    neighborhood = col[2].text.strip()

    #If borough is not assigned then no need to populate the values in the data frame - so loop back
    if borough == "Not assigned":
        continue
        
    #If neighborhood is not assigned then populate neighborhood as the borough
    if neighborhood == "Not assigned":
        neighborhood=borough

    #If postal code is same as the last row postal code then combine both neighborhood and continue
    if postalcode == lastpostalcode:
        lastneighborhood = lastneighborhood + ", " + neighborhood
        continue
    
    #No need to write when the first row is being read
    if lastpostalcode=="":
        pass
    #If control reaches here then all values OK and new postal code started - so to be written to the dataframe   
    else:
        neighborhood_data = neighborhood_data.append({"PostalCode":lastpostalcode, "Borough":lastborough, "Neighborhood":lastneighborhood}, ignore_index=True)
        
    #Assign the new values to the last variables
    lastpostalcode=postalcode
    lastborough=borough
    lastneighborhood=neighborhood
pd.set_option('display.max_colwidth',None)    
neighborhood_data

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
...,...,...,...
97,M5X,Downtown Toronto,"First Canadian Place, Underground city"
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern


In [44]:
neighborhood_data.shape

(102, 3)